In [129]:
import pandas as pd

import pickle
import preprocessor as p
import numpy as np
import nltk
import string
from textblob import TextBlob

In [130]:
data = pd.read_csv('../../data/formspring_data_1.csv', sep = '\t')

In [131]:
def get_label(post):
    label = 1 if ((post.ans1 == "Yes" and post.ans2 == "Yes") or (post.ans2 == "Yes" and post.ans3 == "Yes") or (post.ans1 == "Yes" and post.ans3 == "Yes")) else 0
    return label

data['attack'] = data.apply(get_label, axis=1)

In [132]:
data.head()

,userid,post,ques,ans,asker,ans1,severity1,ans2,severity2,ans3,severity3,attack
0,aguitarplayer94,Q: what&#039;s your favorite song? :D<br>A: I ...,what&#039;s your favorite song? :D<br>,I like too many songs to have a favorite,None,No,0,No,0,No,0,0
1,aprilpooh15,Q: <3<br>A: </3 ? haha jk! <33,<3,</3 ? haha jk! <33,None,No,0,No,0,No,0,0
2,aprilpooh15,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,&quot;hey angel you duh sexy&quot;,Really?!?! Thanks?! haha,None,No,0,No,0,No,0,0
3,aprilpooh15,Q: (:<br>A: ;(,(:,;(,None,No,0,No,0,No,0,0
4,aprilpooh15,Q: ******************MEOWWW*******************...,******************MEOWWW*************************,*RAWR*?,None,No,0,No,0,No,0,0


In [133]:
data['post'] = data['post'].apply(lambda x: x.replace('Q:',' ').replace('A:',' ').replace('<br>',' '))
data['comment'] = data['post'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))
data['length'] = data["comment"].apply(lambda x: len(x.split()))

In [134]:
commentlength = data["comment"].apply(lambda x: len(x.split()))
lengthlimit = commentlength.quantile(.95)

table1dict = {'# Posts: ': [len(data)], '# Classes:': [len(data["attack"].unique())], '95% length:': [lengthlimit], "max length:": [commentlength.max()]}

In [135]:
pd.DataFrame(table1dict)

,# Posts:,# Classes:,95% length:,max length:
0,12773,2,58.0,1076


In [136]:
# table 1 info
print("# Posts:", len(data))
print("# Classes:", len(data["attack"].unique()))
commentlength = data["comment"].apply(lambda x: len(x.split()))
lengthlimit = commentlength.quantile(.95)
print("95% length:", lengthlimit)
print("max length:", commentlength.max())

# Posts: 12773
# Classes: 2
95% length: 58.0
max length: 1076


In [137]:
data.to_csv('../../data/formspring_data_preprocessed.csv', index=False)

In [138]:
data = data[['comment','asker','attack']]

In [139]:
swear_words = "../../swear_words.txt"
word_list = []
with open(swear_words) as f:
    word_list = f.read().splitlines()

In [140]:
def tokens(post):
    return TextBlob(post).words
    
def get_bad_word_count(post):
    x = tokens(post)
    count = 0
    bad_words = []
    for word in x:
        if word in word_list:
            count+=1
            bad_words.append(word)
    return count, str(bad_words)

data['bad_word_count'],data['bad_word_list']= zip(*data['comment'].map(get_bad_word_count)) 

In [141]:
bwc, bwl = zip(*data["comment"].map(get_bad_word_count))
has_bw = pd.Series(np.array(bwc) > 0, index=data.index)
has_swears = data["comment"].apply(lambda x: any([swear in x for swear in word_list]))
swears = pd.concat([has_bw, has_swears], axis=1, keys=["original", "new"])

In [142]:
def data_analysis(comments):
    total = len(comments)
    bully = len(comments[comments['attack']==1])
    swear = len(comments[comments['bad_word_count']>0])
    anonymous = len(comments[comments['asker']=='None'])
    swear_bully = len(comments[(comments['bad_word_count']>0) & (comments['attack']==1) ])
    anonymous_bully = len(comments[(comments['attack']==1) & (comments['asker']=='None')])
    anonymous_swear = len(comments[(comments['bad_word_count']>0)  & (comments['asker']=='None')])
    anonymous_bully_swear = len(comments[(comments['attack']==1) & (comments['bad_word_count']>0)  & (comments['asker']=='None')])
    
    P_B = float(bully)/total
    P_S = float(swear)/total
    P_A = float(anonymous)/total
    P_B_S = float(swear_bully)/swear
    P_S_B = float(swear_bully)/bully
    P_B_A = float(anonymous_bully)/anonymous
    P_A_B = float(anonymous_bully)/bully
    P_S_A = float(anonymous_swear)/anonymous
    P_A_S = float(anonymous_swear)/total
    P_B_A_S = float(anonymous_bully_swear)/ anonymous_swear
    
    probs = pd.DataFrame({'P(B):': [round(P_B,2)],
                        'P(S):': [round(P_S, 2)],
                        'P(A):': [round(P_A, 2)],
                        'P(B|S):': [round(P_B_S, 2)],
                        'P(S|B):':[round( P_S_B, 2)],
                        'P(B|A):':[round(P_B_A, 2)],
                        'P(A|B):': [round(P_A_B, 2)],
                        'P(S|A):': [round(P_S_A, 2)],
                        'P(B|A & S)': [round(P_B_A_S, 2)]})
    return probs

In [143]:
probs = data_analysis(data)

In [144]:
probs

,P(B):,P(S):,P(A):,P(B|S):,P(S|B):,P(B|A):,P(A|B):,P(S|A):,P(B|A & S)
0,0.06,0.16,0.53,0.22,0.59,0.08,0.71,0.2,0.25
